In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
cols = ['PEEDUCA', 'EDUC', 'BASICWGT', 'AGE', 'MONTH', 'YEAR', 'LFS', 'HRSACTT', 'FEMALE']

educdt = lambda x: np.where((x.EDUC == 'SC') & (x.PEEDUCA == 40), 'SCND', 
                   np.where((x.EDUC == 'SC') & (x.PEEDUCA == 41), 'VOC',
                   np.where((x.EDUC == 'SC') & (x.PEEDUCA == 42), 'AAD', x.EDUC)))

df = cps_12mo(cps_dir, cps_date(), cols).query('AGE > 24').assign(EDUCDT = educdt)
date2000 = pd.to_datetime('2000-12-01')
df2 = cps_12mo(cps_dir, date2000, cols).query('AGE > 24').assign(EDUCDT = educdt)

df3 = df.query('LFS == "Employed"')
df4 = df2.query('LFS == "Employed"')

data = pd.DataFrame()

data['2000'] = df2.groupby('EDUCDT').BASICWGT.sum() / df2.BASICWGT.sum() * 100
data['latest'] = df.groupby('EDUCDT').BASICWGT.sum() / df.BASICWGT.sum() * 100

data['2000_emp'] = df4.groupby('EDUCDT').BASICWGT.sum() / df4.BASICWGT.sum() * 100
data['latest_emp'] = df3.groupby('EDUCDT').BASICWGT.sum() / df3.BASICWGT.sum() * 100

data = data.loc[['LTHS', 'HS', 'SCND', 'VOC', 'AAD', 'COLL', 'ADV']]

data.index = ['No High School Diploma', 'High School Diploma', 'Some College, No Degree', 
              'Associate Degree,\\\*Vocational',
              'Associate Degree,\\\*Academic', "Bachelor's Degree", 'Advanced Degree']

data.to_csv(data_dir / 'cps_educ.csv', sep=';', index_label='name')

In [3]:
tot = (df.groupby('EDUC').BASICWGT.sum() / 12000000).loc[['LTHS', 'HS', 'SC', 'COLL', 'ADV']]

tot.index = ['No High School Diploma', 'High School Diploma', 'Some College or Associate Degree', 
             "Bachelor's Degree", 'Advanced Degree']

tot.to_csv(data_dir / 'cps_educ_tot.csv', index_label='name', header=True)

ltdate = dtxt(cps_date())['mon1']

ba_adv_tot = tot["Bachelor's Degree"] + tot['Advanced Degree']
ba_adv_sh = data.loc["Bachelor's Degree", 'latest'] + data.loc['Advanced Degree', 'latest']

adv_tot = tot['Advanced Degree']
adv_sh = data.loc['Advanced Degree', 'latest']

sc_tot = tot['Some College or Associate Degree']

hs_tot = tot['High School Diploma']

lths_tot = tot['No High School Diploma']

text = (f'Over the year ending {ltdate}, {ba_adv_tot:.1f} million '+
        f'people over the age of 25, or {ba_adv_sh:.1f} percent of the total, '+
        f"have at least a bachelor's degree, with {adv_tot:.1f} million of "+
        f'those, or {adv_sh:.1f} percent of the total, holding '+
        "an advanced degree such as a master's degree, medical or law degree, or PhD. "+
        f'An additional {sc_tot:.1f} million people have some college coursework '+
        f'but no degree or have an associate degree. A total of {hs_tot:.1f} million '+
        f'have a high school diploma but no college, while {lths_tot:.1f} million '+
        f'have no high school diploma.')
write_txt(text_dir / 'cps_educ.txt', text)
write_txt(text_dir / 'cps_ltdate.txt', ltdate)

text

"Over the year ending April 2020, 80.9 million people over the age of 25, or 36.5 percent of the total, have at least a bachelor's degree, with 30.3 million of those, or 13.6 percent of the total, holding an advanced degree such as a master's degree, medical or law degree, or PhD. An additional 57.7 million people have some college coursework but no degree or have an associate degree. A total of 62.3 million have a high school diploma but no college, while 21.1 million have no high school diploma."

In [4]:
ba_adv_sh_pr = data.loc["Bachelor's Degree", '2000'] + data.loc['Advanced Degree', '2000']

ba_adv_sh_ch = ba_adv_sh - ba_adv_sh_pr

ba_adv_sh_emp = data.loc["Bachelor's Degree", 'latest_emp'] + data.loc['Advanced Degree', 'latest_emp']

ba_adv_sh_emp_pr = data.loc["Bachelor's Degree", '2000_emp'] + data.loc['Advanced Degree', '2000_emp']
ba_adv_sh_emp_ch = ba_adv_sh_emp - ba_adv_sh_emp_pr

text = ("The share of the population with a bachelor's degree or advanced degree "+
        f"increased by {ba_adv_sh_ch:.1f} percentage points since 2000. The increase "+
        "is even more pronounced among those who are employed; "+
        f"{ba_adv_sh_emp:.1f} percent have a college degree or advanced degree in {ltdate}, an "+
        f"increase of {ba_adv_sh_emp_ch:.1f} percentage points since 2000. ")

write_txt(text_dir / 'cps_educ2.txt', text)

write_txt(text_dir / 'cps_ltdate.txt', ltdate)